In [15]:
# Import libraries
import numpy as np  
import pandas as pd  
import matplotlib.pyplot as plt
import re
import string
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle
from sklearn.preprocessing import StandardScaler
import math
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import warnings
warnings.filterwarnings("ignore")
nltk.download('stopwords')
import random
import faiss
import time
from pynndescent import NNDescent
import elasticsearch

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\paizc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
# Read the dataset
data = pd.read_csv('Reviews.csv')

In [17]:
# Set of stopwords
stop = set(stopwords.words('english'))

# Initialising the snowball stemmer
sno = nltk.stem.SnowballStemmer('english') 

# Function to clean the word of any html-tags
def cleanhtml(sentence): 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', sentence)
    return cleantext

# Function to clean the word of any punctuation or special characters
def cleanpunc(sentence): 
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return  cleaned

i=0
str1=' '
final_string=[]

# Store words from +ve reviews 
all_positive_words=[]

# Store words from -ve reviews
all_negative_words=[] 
s=''

final_string=[]
all_positive_words=[] 
all_negative_words=[] 
s=''
for sent in data['Text'].values:
    filtered_sentence=[]

    # Remove HTMl tags
    sent=cleanhtml(sent) 
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if((cleaned_words.isalpha()) & (len(cleaned_words)>2)):    
                if(cleaned_words.lower() not in stop):
                    s=(sno.stem(cleaned_words.lower())).encode('utf8')
                    filtered_sentence.append(s)
                else:
                    continue
            else:
                continue 
    str1 = b" ".join(filtered_sentence) #final string of cleaned words    
    final_string.append(str1)
    i+=1

In [18]:
count_vect = CountVectorizer() 
final_bow_count = count_vect.fit_transform(final_string)

In [19]:
final_bow_np = StandardScaler(with_mean=False).fit_transform(final_bow_count )

In [20]:
X = final_bow_np

X_train =  final_bow_np[:math.ceil(len(data)*.9)] 
X_test = final_bow_np[math.ceil(len(data)*.9):]

In [21]:
print(X_train.shape)
print(X_test.shape)

(18000, 16482)
(2000, 16482)


Annoy

In [28]:
%%time
k = 50

X_train_dense = X_train.toarray()
X_test_dense = X_test.toarray()

yaya = NNDescent(X_train_dense, n_jobs=1)

ValueError: cannot assign slice from input of different size

In [ ]:
distance = np.array(distances)
neighbors = np.array(neighbors)

In [ ]:
def recall(pred, gt):
    """Compute the recall of the pred neighborhood vs. the gt neighborhood"""
    if pred.shape[0] == 1:
        return len(set(pred).intersection(set(gt)))/len(gt)
    assert pred.shape[0] == gt.shape[0]
    return np.array([len(set(pred[i]).intersection(set(gt[i])))/len(gt[i]) for i in range(len(pred))]).mean()

In [ ]:
# recall for a random assignment of neighbors
recall(np.random.randint(len(neighbors), size=(len(neighbors), k)), neighbors)

In [ ]:
# recall if the last 3 neighbors were missed for each sample
recall(neighbors[:, range(k-3)], neighbors)

In [ ]:
# recall if you found all the correct neighbors
recall(neighbors, neighbors)